# Experiments and Runs

## Defining an Experiment

In the DRYTorch framework, an experiment is a fully reproducible execution of code defined entirely by a configuration file. For example, this design implies that:

- a result obtained by modifying the configuration file (e.g., changing the optimizer) constitutes a new experiment instance.

-  a parameter sweep (or grid search), when fully described within the configuration file, is considered a single experiment.

To define an experiment, you should subclass or annotate DRYTorch's Experiment class, specifying the required configuration type. The Experiment class needs an unique name for each instance and also accepts optional tags and a designated output directory for the run, which other framework components will utilize.

In [2]:
import dataclasses

from typing import TypeAlias

from drytorch import Experiment as GenericExperiment


@dataclasses.dataclass(frozen=True)
class SimpleConfig:
    """A simple configuration."""

    batch_size: int


class MySubclassedExperiment(GenericExperiment[SimpleConfig]):
    """Class for Simple Experiments."""


# alternatively:
MyAnnotatedExperiment: TypeAlias = GenericExperiment[SimpleConfig]

my_config = SimpleConfig(32)
my_experiment = MySubclassedExperiment(
    my_config,
    name='SimpleExp',
    par_dir='.',
    tags=[],
)

# Starting a Run
In the DRYTorch framework, a run is a single execution instance of an experiment's code. Multiple runs of the same experiment—for example, by varying the random seed—are used to replicate and validate results.

You initiate a run instance using the Experiment's create_run method. This instance serves as a context manager (with block) for the experiment's execution code.

The run's ID is a timestamp by default, but you can specify a unique, descriptive name.

You can resume a run by specifying its unique name in create_run. If a name is not provided, DRYTorch attempts to resume the last recorded run.

Note: DRYTorch maintains a run registry on the local disk to track and manage all run IDs and states.


In [3]:
def implement_experiment() -> None:
    """Here should the code for the experiment."""


with my_experiment.create_run() as run:
    first_id = run.id
    implement_experiment()


with my_experiment.create_run(resume=True) as run:
    second_id = run.id
    implement_experiment()

if first_id != second_id:
    raise

[2025-10-11 22:36:04] - Running experiment: SimpleExp.
[2025-10-11 22:36:04] - Running experiment: SimpleExp.


For convenience, especially in interactive environments like notebooks, you can manually start and stop a run, avoiding the context manager.

To do this, use the Experiment's start_run() method and ensure you explicitly call run.stop() when finished.

Warning: If you forget to call run.stop(), the run may not be properly recorded or finalized. While DRYTorch uses weak references to attempt cleanup at the end of the session, this behavior is unreliable and should not be depended upon for correct run logging.

In [4]:
run = my_experiment.create_run()
run.start()
run.stop()

[2025-10-11 22:36:04] - Running experiment: SimpleExp.


## Trackers
The previous output contains a log generated when starting the run. The Experiment class allows you to fully customize which trackers will be active for the experiment, letting you replace or augment the default logging behavior (see the notebook about trackers for more detail).

In [5]:
my_experiment.trackers.remove('BuiltinLogger')

with my_experiment.create_run(resume=True) as run:
    second_id = run.id
    implement_experiment()

# no output

## Global configuration

It is possible to access the configuration file directly from the Experiment class when the a run is on. Otherwise, this operation will fail.

In [6]:
from drytorch.core import exceptions


def get_batch() -> int:
    """Retrieve the batch size setting."""
    return MySubclassedExperiment.get_config().batch_size


with my_experiment.create_run():
    get_batch()

try:
    get_batch()
except (exceptions.AccessOutsideScopeError, exceptions.NoActiveExperimentError):
    err_str = 'Configuration accessed when no run is on.'
else:
    err_str = ''


err_str

'Configuration accessed outside the run'

DRYTorch discourages information leakage between runs to ensure reproducibility.

The framework explicitly prevents constructing a Model instance based on a module registered in a previous run. This isolation ensures that each run starts from a clean state defined solely by its configuration.

In [ ]:
from torch.nn import Linear

from drytorch import Model


my_second_exp = MyAnnotatedExperiment(my_config)

with my_experiment.create_run():
    first_model = Model(Linear(1, 1))

with my_second_exp.create_run():
    second_model = Model(first_model.module)

[2025-10-11 22:36:04] - Running experiment: Experiment.
